### Workbook for estimating population counts in villages surrounding Rwanda bridges
Week of April 14, 2025
Author: Adele Birkenes

This workbook reads in village boundaries and bridge locations and creates a dataset of villages that are within 2 km of at least one bridge. The dataset associates each of these villages with a list of the bridges it is within 2 km of (often it is just one). It then extracts population values for each village from the WorldPop 2020 raster dataset.

Step 1: Import packages & read in data on village boundaries, population (WorldPop raster), and bridges

In [ ]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None)
from shapely.geometry import Point, LineString, Polygon
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os
import numpy as np

In [ ]:
path = "../../synced-data/population-exploration/"

# Village boundaries (geodataframe)
village_boundaries_filename = os.path.join(path,"Rwanda Village Boundaries/Village.shp")
Rwanda_village_boundaries = gpd.read_file(os.path.join(path, village_boundaries_filename))

# Bridges (dataframe)
bridges_filename = os.path.join(path, "rct-all-bridges.csv")
Rwanda_bridges = pd.read_csv(os.path.join(path, bridges_filename))

# Population
rwa_pop_2020_filename = os.path.join(path, "rwa_ppp_2020.tif")

Step 2: Convert bridges dataframe to geodataframe & match projection to that of village boundaries

In [ ]:
def map_bridges(bridges, village_boundaries):

    # Check CRS of village boundaries gdf
    print(f'The CRS of the village boundaries gdf is: {village_boundaries.crs}')

    # Create lat/lon variables
    lon = bridges['GPS (Longitude)']
    lat = bridges['GPS (Latitude)']

    # Create gdf of bridges data by converting lat/lon values to list of Shapely Point objects
    bridge_points = gpd.GeoDataFrame(bridges, geometry=gpd.points_from_xy(x=lon, y=lat), crs='EPSG:4326')

    # Set CRS of bridges gdf to CRS of village boundaries gdf
    bridge_points.to_crs(village_boundaries.crs, inplace=True)

    # Check that reprojection was successful
    print(f'The CRS of the bridges gdf is: {bridge_points.crs}')
    
    return bridge_points

bridge_points = map_bridges(bridges = Rwanda_bridges, village_boundaries = Rwanda_village_boundaries)

Step 3: Create buffers around bridges according to user's distance input

In [ ]:
def create_bridge_buffers(bridges, buffer_distance):

    # Create buffers around bridges according to user's distance input
    buf = bridges.geometry.buffer(distance=buffer_distance)

    # Add bridges point attribute information to buffers
    bridge_buffers = bridge_points.copy()
    bridge_buffers['geometry'] = buf

    return bridge_buffers

bridge_buffers = create_bridge_buffers(bridges = bridge_points, buffer_distance = 2000)
bridge_buffers.head()

Step 4: Perform a spatial join to identify villages that overlap with the bridge buffers and return a df listing each village and its associated bridge IDs

In [ ]:
def associate_villages_with_bridges(village_gdf, bridge_buffers_gdf, village_id_col, bridge_id_col):

    # Perform spatial join
    joined = gpd.sjoin(village_gdf, bridge_buffers_gdf, how="inner", predicate="intersects")

    # Group bridge IDs by village ID
    grouped = joined.groupby(village_id_col)[bridge_id_col].apply(list).reset_index()

    # Drop duplicates so each village ID appears only once
    village_attrs = village_gdf.drop_duplicates(subset=village_id_col)

    # Merge grouped bridge lists back into village data
    result = village_attrs.merge(grouped, on=village_id_col, how='left')

    # Drop villages with no nearby bridges (i.e. CaseSafeID is null)
    result = result.dropna(subset=[bridge_id_col])

    return result

village_bridge_gdf = associate_villages_with_bridges(village_gdf = Rwanda_village_boundaries, 
                                                    bridge_buffers_gdf = bridge_buffers, 
                                                    village_id_col='Village_ID', 
                                                    bridge_id_col='CaseSafeID')

village_bridge_gdf.explore()

Step 5: Reproject population raster to match the projection of the village boundaries

In [ ]:
def reproject_raster(population_raster_file, villages_near_bridges):
    
    # Open the raster file
    with rasterio.open(population_raster_file) as src:
        src_crs = src.crs
        dst_crs = villages_near_bridges.crs
        print(f'The starting CRS of the raster is: {src_crs}')
        print(f'The target CRS from villages is: {dst_crs}')

        # Calculate the transform, width, height for the destination raster
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )

        # Update metadata
        meta = src.meta.copy()
        meta.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Prepare an empty destination array
        dest = np.empty((src.count, height, width), dtype=src.dtypes[0])

        # Reproject
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=dest[i - 1],
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest
            )

    # Save to file
    reprojected_raster_file = os.path.join("../../unsynced-data/", "reprojected_population_raster.tif")
    with rasterio.open(reprojected_raster_file, 'w', **meta) as dst:
        dst.write(dest)

    print(f'Reprojected raster saved to: {reprojected_raster_file}')
    
    return dest, transform, meta

reprojected_worldpop = reproject_raster(population_raster_file=rwa_pop_2020_filename,
                                        villages_near_bridges=village_bridge_gdf)

Step 6: Extract population data for each village from WorldPop raster (or any gridded population raster)

In [ ]:
def extract_pop(reprojected_pop_raster, population_raster_name, villages_near_bridges):
    
    # Copy input geodataframe to avoid modifying the original
    villages_copy = villages_near_bridges.copy()
    
    with rasterio.open(reprojected_pop_raster) as src:
        # Print information from raster profile
        print("CRS:", src.crs)
        print("Raster shape:", src.shape)
        print("Number of Bands:", src.count)

        # Iterate over each village and extract population data
        results = []
        for idx, row in villages_copy.iterrows():
            try:
                out_image = mask(src, [row.geometry], crop=True)
                out_image = out_image[0]

                # Handle masked arrays and replace with 0
                if np.ma.is_masked(out_image):
                    out_image = out_image.filled(0)

                # Remove negative values (e.g., nodata placeholders)
                out_image = np.where(out_image < 0, 0, out_image)

                total_population = np.sum(out_image)

            except Exception as e:
                print(f"Error at village {idx}: {e}")
                total_population = 0  # or np.nan

            results.append(total_population)

        # Add population data to the gdf
        villages_copy[population_raster_name] = results

    return villages_copy

# Apply function to extract population within each village
Rwanda_village_pop = extract_pop(reprojected_pop_raster="../../unsynced-data/reprojected_population_raster.tif", 
                                 population_raster_name="WorldPop_2020",
                                 villages_near_bridges=village_bridge_gdf)

Rwanda_village_pop.head()
Rwanda_village_pop.explore()